In [2]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import colors
import random
import sys
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import Dataset

In [ ]:
batch_size=32
device='cpu'
lr=0.0001

In [ ]:
class NN(nn.Module):

    def __init__(self, in_channels):
      super(NN, self).__init__()

      self.first_block = nn.Sequential(
          nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, padding='same'),
          nn.ReLU(inplace=True),
          nn.Conv2d(in_channels=32, out_channels=4, kernel_size=3, padding='same'),
          nn.ReLU(inplace=True),
      )

      self.backbone = nn.Sequential()
      self.classifier = nn.Linear(4 * 28 *28, 10)
      self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
      x = self.first_block(x)
      #x = self.backbone(x)
      x = torch.flatten(x, start_dim=1)
      out = self.classifier(x)
      out = self.softmax(out)
      return out

    def __call__(self, x):
      return self.forward(x)

In [ ]:
class env():
  metadata = {'render_modes': ['human']}

  """
  actions:
      0) Conv2d(x, 2x, 3,3, padding='same') + relu
      1) Conv2d(x, 2x, 5,5, padding='same') + relu
      2) Conv2d(x, 2x, 7,7, padding='same') + relu
      3) Conv2d(2x, x, 3,3, padding='same') + relu
      4) Conv2d(2x, x, 5,5, padding='same') + relu
      5) Conv2d(2x, x, 7,7, padding='same') + relu
      6) dropout(0.1)
      7) dropout(0.2)
      8) dropout(0.4)
      9) MaxPool2D(2)
      10) AvgPool2D(2)
      11) BatchNorm2d()
  """

  def __init__(self, render_mode=None):
      self.max_its = 10
      self.current_it = 1
      self.episode_reward = 0
      #self.last_layer_shape = None
      self.N_final_losses = 10
      self.N_epochs = 10
      self.N_max_layers = 10
      self.N_amount_actions = 12
      self.in_channels = 1
      self.last_step_valid_losses = np.zeros(self.N_epochs)
      self.final_losses = np.zeros(self.N_final_losses)

      self.NN = NN(self.in_channels)
      self.train_dataloader = None
      self.train_dataloader = None
      self.optimizer = None
      self.criterion = None

      self.action_space = spaces.MultiDiscrete([self.N_amount_actions] * self.N_max_layers)

      self.observation_space = spaces.Dict(
      {
          'last_final_losses': spaces.Box(low=0, high=2, shape=(self.N_final_losses,)),
          'last_step_valid_losses': spaces.Box(low=0, high=2, shape=(self.N_epochs,)),
      }
      )
      self.statistics = {
        'episode_rewards': [],
        'global_rewards': [],
        'made_steps': [],
      }
      self.seed()

      assert render_mode is None or render_mode in self.metadata["render_modes"]
      self.render_mode = render_mode



  def set_train_dataloader(self, train_dataloader):
    self.train_dataloader = train_dataloader

  def set_valid_dataloader(self, valid_dataloader):
    self.valid_dataloader = valid_dataloader

  def set_criterion(self, criterion):
    self.criterion = criterion

  def set_optimizer(self, optimizer):
    self.optimizer = optimizer

  def train(self):
    train_losses = []
    valid_losses = []

    def CalcValLoss(self):
        with torch.no_grad():
            losses = []
            for X, Y in self.valid_dataloader:
                X = X.float().to(device)
                Y = Y.float().to(device)
                preds = self.NN(X)
                preds, _ = torch.max(preds,1)
                loss = self.criterion(preds,Y)
                losses.append(loss.item())
            print("Valid Loss : {:.6f}".format(torch.tensor(losses).mean()))
            valid_losses.append(torch.tensor(losses).mean())

    for i in range(1, self.N_epochs+1):
        losses = []
        for X, Y in tqdm(self.train_dataloader):
            X = X.float().to(device)
            Y = Y.float().to(device)
            preds = self.NN(X)
            preds, _ = torch.max(preds,1)
            loss = self.criterion(preds, Y)
            losses.append(loss.item())

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        print("Train Loss : {:.6f}".format(torch.tensor(losses).mean()))
        train_losses.append(torch.tensor(losses).mean())
    return train_losses, valid_losses


  def seed(self, seed=None):
      self.np_random, seed = seeding.np_random(seed)
      return [seed]

  def action_to_NN(self, action, last_out_shape): # last shape = [BS,C,H,W]
    layer = None
    C = last_out_shape[1]
    if (action == 1): # Conv2d(x, 2x, 3,3, padding='same') + relu
      layer = nn.Sequential(
          nn.Conv2d(C, 2*C, kernel_size=3, padding='same'),
          nn.ReLU(inplace=True),
      )
    elif (action == 2): # Conv2d(x, 2x, 5,5, padding='same') + relu
      layer = nn.Sequential(
          nn.Conv2d(C, 2*C, kernel_size=5, padding='same'),
          nn.ReLU(inplace=True),
      )
    elif (action == 3): # Conv2d(x, 2x, 7,7, padding='same') + relu
      layer = nn.Sequential(
          nn.Conv2d(C, 2*C, kernel_size=7, padding='same'),
          nn.ReLU(inplace=True),
      )
    elif (action == 4): # Conv2d(x, 2x, 3,3, padding='same') + sigmoid
      layer = nn.Sequential(
          nn.Conv2d(C, 2*C, kernel_size=3, padding='same'),
          nn.Sigmoid(),
      )
    elif (action == 5): # Conv2d(x, 2x, 5,5, padding='same') + sigmoid
      layer = nn.Sequential(
          nn.Conv2d(C, 2*C, kernel_size=5, padding='same'),
          nn.Sigmoid(),
      )
    elif (action == 6): # Conv2d(x, 2x, 7,7, padding='same') + sigmoid
      layer = nn.Sequential(
          nn.Conv2d(C, 2*C, kernel_size=7, padding='same'),
          nn.Sigmoid(),
      )

    elif (action == 7): # Conv2d(2x, x, 3,3, padding='same') + relu
      layer = nn.Sequential(
          nn.Conv2d(C, C/2, kernel_size=3, padding='same'),
          nn.ReLU(inplace=True),
      )
    elif (action == 8): # Conv2d(2x, x, 5,5, padding='same') + relu
      layer = nn.Sequential(
          nn.ReLU(inplace=True),
      )
    elif (action == 9): # Conv2d(2x, x, 7,7, padding='same') + relu
      layer = nn.Sequential(
          nn.ReLU(inplace=True),
      )
    elif (action == 10): # Conv2d(2x, x, 3,3, padding='same') + sigmoid
      layer = nn.Sequential(
          nn.Sigmoid(),
      )
    elif (action == 11): # Conv2d(2x, x, 5,5, padding='same') + sigmoid
      layer = nn.Sequential(
          nn.Sigmoid(),
      )
    elif (action == 12): # Conv2d(2x, x, 7,7, padding='same') + sigmoid
      layer = nn.Sequential(
          nn.Sigmoid(),
      )

    elif (action == 13): # dropout(0.1)
      layer = nn.Sequential(
          nn.Dropout(p=0.1),
      )
    elif (action == 14): # dropout(0.2)
      layer = nn.Sequential(
          nn.Dropout(p=0.2),
      )
    elif (action == 15): # dropout(0.4)
      layer = nn.Sequential(
          nn.Dropout(p=0.4),
      )

    elif (action == 16): # MaxPool2D(2)
      layer = nn.Sequential(
          nn.MaxPool2d(2),
      )
    elif (action == 17): # AvgPool2D(2)
      layer = nn.Sequential(
          nn.AvgPool2d(2),
      )

    elif (action == 18): # BatchNorm2d()
      layer = nn.Sequential(
          nn.BatchNorm2d(C),
      )

    return layer

  def rebuild_nn(self):
    pass

  def step(self, action):
      reward = 0
      done = False
      info = {} # You may need to add some extra information
      '''
      Do some simulation
      '''

      current_obs = {
        'visible_space': None,
        'distance': None,
        'memory': self.last_rewards,
        }
      return current_obs, reward, done, info


  def create_basic_nn(self):
    pass

  def reset(self):
      '''
      Reset the env
      '''
      self.last_rewards = np.zeros(self.N_last_rewards)

      current_obs = {
        'visible_space': self.current_space,
        'distance': self.distance,
        'memory': self.last_rewards}
      self.episode_reward = 0
      self.current_it = 1
      self.statistics['episode_rewards'] = []
      self.statistics['made_steps'] = []

      return current_obs



  def render(self, mode=None):
    pass

  def _render_frame(self):
    pass


  def close(self):
      pass

In [ ]:
env = env()

In [ ]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [ ]:
train_samples = np.expand_dims(np.array(train_data.data), axis=1)[:5000]

In [ ]:
train_labels = np.array(train_data.targets)[:5000]

In [ ]:

class myDataset(Dataset):
    def __init__(self, X, y):

      self.X = X
      self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x_ = self.X[idx]
        y_ = self.y[idx]
        return x_, y_

In [ ]:
dataset = myDataset(X=train_samples, y=train_labels)

In [ ]:
train_set, valid_set = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

In [ ]:

train_dataloader = torch.utils.data.DataLoader(
  train_set,
  batch_size=batch_size,
  shuffle=True,
  drop_last=True)

valid_dataloader = torch.utils.data.DataLoader(
  valid_set,
  batch_size=batch_size,
  drop_last=True,
  shuffle=True)

In [ ]:
env.set_train_dataloader(train_dataloader)
env.set_valid_dataloader(valid_dataloader)

In [ ]:
env.set_criterion(torch.nn.CrossEntropyLoss())
env.set_optimizer(torch.optim.SGD(env.NN.parameters(), lr=lr))

In [ ]:
env.train()

100%|██████████| 125/125 [00:01<00:00, 67.06it/s]


Train Loss : 490.178558


100%|██████████| 125/125 [00:02<00:00, 60.31it/s]


Train Loss : 485.136169


100%|██████████| 125/125 [00:01<00:00, 74.70it/s]


...

100%|██████████| 125/125 [00:01<00:00, 74.26it/s]


Train Loss : 476.843506


100%|██████████| 125/125 [00:01<00:00, 77.46it/s]

Train Loss : 476.379364


([tensor(490.1786),
  tensor(485.1362),
  tensor(481.9269),
  tensor(480.3055),
  tensor(479.2868),
  tensor(478.4208),
  tensor(477.7364),
  tensor(477.1236),
  tensor(476.8435),
  tensor(476.3794)],
 [])

In [ ]:
# PyTorch 2D layers with parameters
# conv2d() + relu
# torch.nn.Conv2d(
#     in_channels,
#     out_channels,
#     kernel_size,
#     stride=1,
#     padding=0,
#     dilation=1,
#     groups=1,
#     bias=True,
#     padding_mode='zeros',
#     device=None,
#     dtype=None
#     )

# batchnorm2d()
# torch.nn.BatchNorm2d(
#     num_features,
#     eps=1e-05,
#     momentum=0.1,
#     #affine=True,
#     #track_running_stats=True,
#     #device=None,
#     #dtype=None
#     )

# avgpool2d()
# torch.nn.AvgPool2d(
#     kernel_size,
#     stride=None,
#     padding=0,
#     #ceil_mode=False,
#     #count_include_pad=True,
#     #divisor_override=None
#     )

# maxpool2d()
# torch.nn.MaxPool2d(
#     kernel_size,
#     stride=None,
#     padding=0,
#     dilation=1,
#     return_indices=False,
#     ceil_mode=False)

# dropout()
# torch.nn.Dropout2d(
#     p=0.5,
#     #inplace=False
#     )

In [70]:
t = spaces.Text(max_length=10, min_length=3,charset={'conv=channel_factor:2,kernel_size:3,stride:1,padding:0-',
                                                     'conv=channel_factor:0.5,kernel_size:3,stride:1,padding:0-',
                                                     'batchnorm=eps:0.00001-',
                                                     'avgpool=kernel_size:2,stride:2,padding:0-',
                                                     'avgpool=kernel_size:3,stride:2,padding:0-',
                                                     'maxpool=kernel_size:2,stride:2,padding:0-',
                                                     'maxpool=kernel_size:3,stride:2,padding:0-',
                                                     'dropout=p:0.1-',
                                                     'dropout=p:0.2-',
                                                     })

In [71]:
action = t.sample()

In [72]:
action

'batchnorm=eps:0.00001-maxpool=kernel_size:2,stride:2,padding:0-avgpool=kernel_size:3,stride:2,padding:0-maxpool=kernel_size:2,stride:2,padding:0-'

In [73]:
class actionParser():
  def __init__(self):
    self.text_layers_dict = dict({})
    pass

  def parse_action(self, action):
    if action[-1] == '-':
      action = action[:-1]
    text_layers = action.split('-')
    for text_layer in text_layers:
      tmp = text_layer.split('=')
      layer_name, layer_params = tmp[0], tmp[1].split(',')
      layer_params_dict = dict({})
      for param in layer_params:
        param = param.split(':')
        param_name, param_value = param[0], param[1]
        layer_params_dict[param_name] = param_value
      self.text_layers_dict[layer_name] = layer_params_dict

  def get_text_layers_dict(self):
    return self.text_layers_dict

In [74]:
parser = actionParser()
parser.parse_action(action)
res = parser.get_text_layers_dict()
for r in res.keys():
  print(r, res[r])

batchnorm {'eps': '0.00001'}
maxpool {'kernel_size': '2', 'stride': '2', 'padding': '0'}
avgpool {'kernel_size': '3', 'stride': '2', 'padding': '0'}
